In [65]:
import numpy as np
import pandas as pd
import data_analysis_temp as da
import plotly.express as px
import plotly.graph_objects as go


In [2]:
user_data, top50 = da.load_data()

/Users/leksa/Documents/MIR/TSAI Hackathon/data-visualisation-web/src/data_analysis.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_50 = top_50.append(df, ignore_index=True)
/Users/leksa/Documents/MIR/TSAI Hackathon/data-visualisation-web/src/data_analysis.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_50 = top_50.append(df, ignore_index=True)
/Users/leksa/Documents/MIR/TSAI Hackathon/data-visualisation-web/src/data_analysis.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_50 = top_50.append(df, ignore_index=True)


In [5]:
top50

,id,name,popularity,url,preview_url,artist_ids,artist_names,genres,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,user_id
0,4f8Mh5wuWHOsfXtzjrJB3t,I THINK,73,https://open.spotify.com/track/4f8Mh5wuWHOsfXt...,https://p.scdn.co/mp3-preview/ffabad2f16786a12...,['4V8LLVI7PbaPR0K2TGSxFF'],"['Tyler, The Creator']","['hip hop', 'rap']",0.826,0.579,...,0,0.0801,0.00881,0.000000,0.1290,0.431,121.075,212013,4,22kkfwe46kc2wbdx5z7r2jy2y
1,5hVghJ4KaYES3BFUATCYn0,EARFQUAKE,81,https://open.spotify.com/track/5hVghJ4KaYES3BF...,https://p.scdn.co/mp3-preview/eb91598e2eff6bb1...,['4V8LLVI7PbaPR0K2TGSxFF'],"['Tyler, The Creator']","['hip hop', 'rap']",0.554,0.498,...,1,0.0685,0.23000,0.000006,0.7950,0.413,79.635,190067,4,22kkfwe46kc2wbdx5z7r2jy2y
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),85,https://open.spotify.com/track/7KA4W4McWYRpgf0...,https://p.scdn.co/mp3-preview/e6c4c9656721fe0a...,"['4V8LLVI7PbaPR0K2TGSxFF', '1U1el3k54VvEUzo3yb...","['Tyler, The Creator', 'Kali Uchis']","[['hip hop', 'rap'], ['colombian pop', 'dance ...",0.558,0.559,...,1,0.0959,0.37100,0.000007,0.1090,0.620,78.558,180387,4,22kkfwe46kc2wbdx5z7r2jy2y
3,7y8X0Z04gJCKtfrnSAMywJ,Hacker,57,https://open.spotify.com/track/7y8X0Z04gJCKtfr...,https://p.scdn.co/mp3-preview/acfc6ea5de718306...,['5RADpgYLOuS2ZxDq7ggYYH'],['Death Grips'],"['alternative hip hop', 'escape room', 'experi...",0.681,0.880,...,1,0.2270,0.05670,0.000000,0.0893,0.380,124.885,275520,4,22kkfwe46kc2wbdx5z7r2jy2y
4,0HAqq2GcQKyi3s87GuN7jU,Amoeba,72,https://open.spotify.com/track/0HAqq2GcQKyi3s8...,https://p.scdn.co/mp3-preview/0f23d3bf01c9c780...,['3l0CmX0FuQjFxr8SK7Vqag'],['Clairo'],"['bedroom pop', 'indie pop', 'pop']",0.739,0.491,...,1,0.0270,0.79100,0.036600,0.1000,0.707,111.008,228922,4,22kkfwe46kc2wbdx5z7r2jy2y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,29m79w9xPMH4YCD6r8JSmV,Excuses,82,https://open.spotify.com/track/29m79w9xPMH4YCD...,https://p.scdn.co/mp3-preview/18e50eaa91327faa...,"['6LEG9Ld1aLImEFEVHdWNSB', '5DHi2MeoRgAwPE0A0q...","['AP Dhillon', 'Gurinder Gill', 'Intense']","[['punjabi hip hop', 'punjabi pop'], [], ['des...",0.837,0.717,...,0,0.0822,0.08420,0.000000,0.1490,0.494,94.984,176233,4,31opghsdw4vzh76ui6mvzo4bdy5i
146,6G699QfN1BWYWXfiblH51F,"Kusu Kusu (From ""Satyameva Jayate 2"")",70,https://open.spotify.com/track/6G699QfN1BWYWXf...,https://p.scdn.co/mp3-preview/a44ddc5b290702a9...,"['4f7KfxeHq9BiylGmyXepGt', '7C5xVXoqim1TICnCMZ...","['Tanishk Bagchi', 'Zahrah S Khan', 'Dev Negi']","[['desi pop', 'modern bollywood'], [], ['desi ...",0.742,0.911,...,1,0.0572,0.01370,0.000810,0.0593,0.780,101.000,195616,4,31opghsdw4vzh76ui6mvzo4bdy5i
147,2O3CJdn2fUh90n7his0v4D,Bilionera,46,https://open.spotify.com/track/2O3CJdn2fUh90n7...,https://p.scdn.co/mp3-preview/2b07df2d29232254...,['6RQDTlies3nrNDJwXvbBZT'],['Otilia'],['romanian pop'],0.666,0.816,...,0,0.0514,0.04640,0.004020,0.1140,0.834,170.169,185202,4,31opghsdw4vzh76ui6mvzo4bdy5i
148,4vwPirpSJCOiIsgxMV4Xcv,Liggi,69,https://open.spotify.com/track/4vwPirpSJCOiIsg...,https://p.scdn.co/mp3-preview/46a9dc6c12e96121...,['72beYOeW2sb2yfcS4JsRvb'],['Ritviz'],"['desi hip hop', 'desi pop', 'hindi indie', 'i...",0.786,0.667,...,1,0.0540,0.01120,0.053000,0.0740,0.688,102.046,181176,4,31opghsdw4vzh76ui6mvzo4bdy5i


### User Based Analysis

In [48]:
qualitative = ['artist_names','genres','key','mode','speechiness','liveness','user_id']
quantative_normalizable = ['popularity', 'danceability', 'energy', 'loudness','speechiness',
       'acousticness', 'instrumentalness', 'valence', 'mode']
quantative_not_normalizable = ['tempo', 'duration_ms','user_id']
# relative way of normalizing - more distinctive result / shows more persona
quantative_normalized = top50[quantative_normalizable].apply(lambda x: (x - x.mean())/x.std(), axis = 0)
# absolute way of normalizing - range is [0,1], less distinctive result / a couple of features are similar among users: energy, loudness, valence, etc. 
# quantative_normalized = (top50[quantative_normalizable] - top50[quantative_normalizable].min())/(top50[quantative_normalizable].max()-top50[quantative_normalizable].min())
quantative_top50 = pd.concat([top50[quantative_not_normalizable],quantative_normalized], axis = 1 )

In [49]:
quantative_normalized

,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,mode
0,0.930488,0.919030,-0.686598,-0.719806,-0.166211,-0.703034,-0.327704,-0.337011,-0.919877
1,1.275113,-1.269393,-1.141899,-0.928946,-0.309135,0.271048,-0.327663,-0.424059,1.079855
2,1.447426,-1.237210,-0.799018,-1.048071,0.028462,0.891987,-0.327652,0.576992,1.079855
3,0.241238,-0.247592,1.005324,0.734467,1.643751,-0.492135,-0.327704,-0.583646,1.079855
4,0.887410,0.219057,-1.181247,-1.815363,-0.820458,2.741593,-0.074946,0.997723,1.079855
...,...,...,...,...,...,...,...,...,...
145,1.318191,1.007533,0.089100,-0.555171,-0.140337,-0.371030,-0.327704,-0.032343,-0.919877
146,0.801254,0.243194,1.179575,1.096864,-0.448363,-0.681499,-0.322110,1.350751,1.079855
147,-0.232622,-0.368277,0.645580,0.752871,-0.519825,-0.537494,-0.299942,1.611894,-0.919877
148,0.758175,0.597203,-0.191950,-0.730179,-0.487790,-0.692509,0.038312,0.905839,1.079855


In [57]:
user_average_data = quantative_top50.groupby(["user_id"]).mean()
user_average_data = user_average_data.join(top50[["user_id", "artist_names","genres"]].groupby(["user_id"]).nunique(), on = 'user_id')
# using 3 criteria: 
# 1. popularity
# 2. dance music lover: 0.6 * danceability + 0.3 * loudness + 0.3 * energy - 0.2 * acousticness
# 3. musical positiveness: 0.8 * valence + 0.2 * mode
# mode -> 1: major key, mode -> 0: minor key
user_average_data["dance_music_lover"] = user_average_data["danceability"] * 0.6 + user_average_data["loudness"] * 0.3 + user_average_data["energy"] * 0.3 - user_average_data["acousticness"] * 0.2
user_average_data["musical_positiveness"] = user_average_data["valence"] * 0.8 + user_average_data["mode"] * 0.2




'user_id'

In [71]:

fig = px.scatter_3d(user_average_data, x = 'dance_music_lover', y = 'musical_positiveness', z = 'popularity', color = 'tempo', size = 'genres', size_max = 18,
                    opacity = 0.7)

fig.update_layout(margin = dict(l=0, r=0, b=0, t=0))
